In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 3, Finished, Available)

In [2]:
%%sql
create table if not exists project_lakehouse.gold_fact_sale
(
    Order_ID string,
    Price float,
    Quantity float,
    Sales float,
    Discount float,
    Profit float,
    Shipping_cost float,
    Order_Date date,
    Shipping_Date date,
    Product_ID long,
    Order_Priority_ID long,
    Ship_mode_ID long,
    Customer_ID string,
    Order_year int,
    Order_Month int,
    Create_TS timestamp,
    Modified_TS timestamp 


)
using DELTA
PARTITIONED by (Order_year,Order_Month)


StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_date = spark.sql(" select coalesce(max(Modified_TS), '1900-01-01') from project_lakehouse.gold_fact_sale").first()[0]

StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 5, Finished, Available)

In [4]:
df_bronze = spark.sql(""" select
    bs.Order_ID,
    bs.Sales as Price,
    bs.Quantity,
    bs.Sales * bs.Quantity as Sales,
    bs.Discount,
    bs.Profit,
    bs.Shipping_Cost,
    bs.Order_Date,
    bs.Shipping_Date,
    gp.Product_ID,
    go.Order_Priority_ID,
    gs.Ship_Mode_ID,
    bs.Customer_ID,
    Year(Order_Date) as Order_Year,
    Month(Order_Date) as Order_Month
from project_lakehouse.bronze_sales as bs
inner join project_lakehouse.gold_product as gp  
        on bs.Product = gp.Product
        and bs.Product_Category = gp.Product_Category
inner join project_lakehouse.gold_shipmode as gs
        on bs.ship_mode = gs.ship_mode
inner join project_lakehouse.gold_order_priority as go
        on bs.order_priority = go.order_Priority
""")

StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 6, Finished, Available)

In [5]:
df_bronze.createOrReplaceTempView("ViewFactSale")

StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 7, Finished, Available)

In [9]:
%%sql

merge into project_lakehouse.gold_fact_sale as gsf
using ViewFactSale as vfs
    on gsf.order_year = vfs.order_year
    and gsf.order_month = vfs.order_month
    and gsf.Order_ID = vfs.Order_ID
when matched then
update SET
gsf.Sales = vfs.Sales,
gsf.Price = vfs.Price,
gsf.Quantity = vfs.Quantity,
gsf.Discount = vfs.Discount,
gsf.Profit = vfs.Profit,
gsf.Shipping_Cost = vfs.Shipping_Cost,
gsf.Order_Date = vfs.Order_Date,
gsf.Shipping_Date = vfs.Shipping_Date,
gsf.Product_ID = vfs.Product_ID,
gsf.Order_Priority_ID = vfs.Order_Priority_ID,
gsf.Ship_Mode_ID = vfs.Ship_Mode_ID,
gsf.Customer_ID = vfs.Customer_ID,
gsf.Modified_TS = current_timestamp()

when not matched THEN
INSERT
(
    gsf.Order_ID ,
    gsf.Sales ,
    gsf.Price ,
    gsf.Quantity ,
    gsf.Discount ,
    gsf.Profit ,
    gsf.Shipping_Cost ,
    gsf.Order_Date ,
    gsf.Shipping_Date ,
    gsf.Product_ID ,
    gsf.Order_Priority_ID ,
    gsf.Ship_Mode_ID ,
    gsf.Customer_ID ,
    gsf.Order_Year ,
    gsf.Order_Month ,
    gsf.Create_TS ,
    gsf.Modified_TS
)

VALUES
(
    vfs.Order_ID,
    vfs.Sales,
    vfs.Price,
    vfs.Quantity,
    vfs.Discount,
    vfs.Profit,
    vfs.Shipping_Cost,
    vfs.Order_Date,
    vfs.Shipping_Date,
    vfs.Product_ID,
    vfs.Order_Priority_ID,
    vfs.Ship_Mode_ID,
    vfs.Customer_ID,
    vfs.Order_Year,
    vfs.Order_Month,
    current_timestamp() ,
    current_timestamp()
)

StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 11, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [10]:
%%sql
select * from  project_lakehouse.gold_fact_sale limit 10

StatementMeta(, 75447cb7-9f30-4de5-b600-330a994b3578, 12, Finished, Available)

<Spark SQL result set with 10 rows and 17 fields>